## Notebook 4: TTS Workflow

We have the exact podcast transcripts ready now to generate our audio for the Podcast.

In this notebook, we will learn how to generate Audio using  `Kokoro` model first. 

After that, we will use the output from Notebook 3 to generate our complete podcast

Note: Please feel free to extend this notebook with newer models. The above two were chosen after some tests using a sample prompt.

In [10]:
!ollama list


NAME                                                         ID              SIZE      MODIFIED     
hf.co/OuteAI/OuteTTS-0.3-500M-GGUF:Q4_K_S                    76c6be93d29c    391 MB    2 days ago      
hf.co/OuteAI/OuteTTS-0.2-500M-GGUF:Q8_0                      0a6c38a67073    536 MB    3 days ago      
mistral-small:24b                                            8039dd90c113    14 GB     7 days ago      
llama3.2-vision:11b                                          085a1fdae525    7.9 GB    7 days ago      
mistral-nemo:latest                                          994f3b8b7801    7.1 GB    7 days ago      
granite3.2:8b                                                9bcb3335083f    4.9 GB    8 days ago      
phi4-mini:latest                                             60f202f815d7    2.8 GB    8 days ago      
granite3.2-vision:latest                                     3be41a661804    2.4 GB    9 days ago      
phi4:latest                                                  ac896e

Credit: [This](https://colab.research.google.com/drive/1dWWkZzvu7L9Bunq9zvD-W02RFUXoW-Pd?usp=sharing#scrollTo=68QtoUqPWdLk) Colab was used for starter code


We can install these packages for speedups

!for KoKoro, you need to follow this if the base

* 1️⃣ Install kokoro
!pip install -q kokoro>=0.8.2 soundfile
* 2️⃣ Install espeak, used for English OOD fallback and some non-English languages
!apt-get -qq -y install espeak-ng > /dev/null 2>&1
or
brew install espeak-ng   # in MacOS

if you face any issue regarding to thinc as a dependency for packages like spaCy
    * Install Python 3.12.6.​
    * Create and activate a virtual environment.​
    * Install thinc or spaCy within this environment.​

Install with Build Isolation Disabled: If you prefer to use Python 3.13, you can try installing thinc with build isolation disabled:

    Ensure numpy is installed:​

pip install numpy

Install thinc without build isolation:​

    'pip install --no-build-isolation thinc'

Use Conda for Installation: Conda can handle dependencies more effectively on macOS:

    'Create and activate a conda environment:​'

conda create -n myenv python=3.12
conda activate myenv

Install thinc using conda:​

        'conda install -c conda-forge thinc'

For more detailed information on installing thinc, refer to the official installation guide. ​
thinc.ai

If you continue to experience issues, consider consulting the spaCy GitHub discussions or Stack Overflow for community support. ​



Let's import the necessary frameworks

In [128]:
from IPython.display import Audio
import IPython.display as ipd
from tqdm import tqdm

In [129]:
from transformers import BarkModel, AutoProcessor, AutoTokenizer
import torch
import json
import numpy as np

### Testing the Audio Generation

Let's try generating audio using Kokoro model to understand how they work. 


Please set `device = "cuda"` below if you're using a single GPU node.

#### Kokoro Model

Let's try using the Parler Model first and generate a short segment with speaker Laura's voice

In [ ]:
# 1️⃣ Install kokoro
# !pip install -q kokoro>=0.8.2 soundfile
# 2️⃣ Install espeak, used for English OOD fallback and some non-English languages
# !apt-get -qq -y install espeak-ng > /dev/null 2>&1
# 🇪🇸 'e' => Spanish es
# 🇫🇷 'f' => French fr-fr
# 🇮🇳 'h' => Hindi hi
# 🇮🇹 'i' => Italian it
# 🇧🇷 'p' => Brazilian Portuguese pt-br

# 3️⃣ Initalize a pipeline
from kokoro import KPipeline
from IPython.display import display, Audio
import soundfile as sf
# 🇺🇸 'a' => American English, 🇬🇧 'b' => British English
# 🇯🇵 'j' => Japanese: pip install misaki[ja]
# 🇨🇳 'z' => Mandarin Chinese: pip install misaki[zh]
pipeline = KPipeline(lang_code='a') # <= make sure lang_code matches voice

# This text is for demonstration purposes only, unseen during training
text = '''
The sky above the port was the color of television, tuned to a dead channel.
"It's not like I'm using," Case heard someone say, as he shouldered his way through the crowd around the door of the Chat. "It's like my body's developed this massive drug deficiency."
It was a Sprawl voice and a Sprawl joke. The Chatsubo was a bar for professional expatriates; you could drink there for a week and never hear two words in Japanese.

These were to have an enormous impact, not only because they were associated with Constantine, but also because, as in so many other areas, the decisions taken by Constantine (or in his name) were to have great significance for centuries to come. One of the main issues was the shape that Christian churches were to take, since there was not, apparently, a tradition of monumental church buildings when Constantine decided to help the Christian church build a series of truly spectacular structures. The main form that these churches took was that of the basilica, a multipurpose rectangular structure, based ultimately on the earlier Greek stoa, which could be found in most of the great cities of the empire. Christianity, unlike classical polytheism, needed a large interior space for the celebration of its religious services, and the basilica aptly filled that need. We naturally do not know the degree to which the emperor was involved in the design of new churches, but it is tempting to connect this with the secular basilica that Constantine completed in the Roman forum (the so-called Basilica of Maxentius) and the one he probably built in Trier, in connection with his residence in the city at a time when he was still caesar.

[Kokoro](/kˈOkəɹO/) is an open-weight TTS model with 82 million parameters. Despite its lightweight architecture, it delivers comparable quality to larger models while being significantly faster and more cost-efficient. With Apache-licensed weights, [Kokoro](/kˈOkəɹO/) can be deployed anywhere from production environments to personal projects.
'''
# text = '「もしおれがただ偶然、そしてこうしようというつもりでなくここに立っているのなら、ちょっとばかり絶望するところだな」と、そんなことが彼の頭に思い浮かんだ。'
# text = '中國人民不信邪也不怕邪，不惹事也不怕事，任何外國不要指望我們會拿自己的核心利益做交易，不要指望我們會吞下損害我國主權、安全、發展利益的苦果！'
# text = 'Los partidos políticos tradicionales compiten con los populismos y los movimientos asamblearios.'
# text = 'Le dromadaire resplendissant déambulait tranquillement dans les méandres en mastiquant de petites feuilles vernissées.'
# text = 'ट्रांसपोर्टरों की हड़ताल लगातार पांचवें दिन जारी, दिसंबर से इलेक्ट्रॉनिक टोल कलेक्शनल सिस्टम'
# text = "Allora cominciava l'insonnia, o un dormiveglia peggiore dell'insonnia, che talvolta assumeva i caratteri dell'incubo."
# text = 'Elabora relatórios de acompanhamento cronológico para as diferentes unidades do Departamento que propõem contratos.'

# 4️⃣ Generate, display, and save audio files in a loop.
generator = pipeline(
    text, voice='af_heart', # <= change voice here
    speed=1, split_pattern=r'\n+'
)
for i, (gs, ps, audio) in enumerate(generator):
    print(i)  # i => index
    print(gs) # gs => graphemes/text
    print(ps) # ps => phonemes
    display(Audio(data=audio, rate=24000, autoplay=i==0))
    sf.write(f'{i}.wav', audio, 24000) # save each audio file


## Bringing it together: Making the Podcast

Okay now that we understand everything-we can now use the complete pipeline to generate the entire podcast

Let's load in our pickle file from earlier and proceed:

In [261]:
import pickle

with open('./resources/podcast_ready_data.pkl', 'rb') as file:
    PODCAST_TEXT = pickle.load(file)

Let's define load in the TTS model and set it's hyper-parameters for discussions

We will concatenate the generated segments of audio and also their respective sampling rates since we will require this to generate the final audio

In [267]:
device="mps"

Function generate text for speaker 1

In [284]:
import random
# Function to determine speed range based on text length relative to average length
def determine_speed_range(text_length, average_length):
    if text_length < average_length:
        return 0.8, .97  # Shorter than average: lower speed range
    else:
        return .97, 1.15  # Average or longer: higher speed range

In [285]:
def generate_speaker1_audio(text,average_length):
    """Generate audio using ParlerTTS for Speaker 1"""
    text_length = len(text)
    # speed = random.uniform(.75, 1.2)
    min_speed, max_speed = determine_speed_range(text_length, average_length)
    speed = random.uniform(min_speed, max_speed)
    print(speed)
    generator = pipeline(
        text, voice='am_liam', # <= change voice here  af_heart am_eric
        speed=speed, split_pattern=r'\n+'
    )
    for i, (gs, ps, audio) in enumerate(generator): 
        # Assuming 'audio' is your tensor
        audio_tensor = audio.cpu().detach().numpy()
        audio_array = np.squeeze(audio_tensor)

    return audio_array,24000

Function to generate text for speaker 2

In [288]:
def generate_speaker2_audio(text,average_length):
    """Generate audio using ParlerTTS for Speaker 1"""
    text_length = len(text)
    # speed = random.uniform(.75, 1.2)
    min_speed, max_speed = determine_speed_range(text_length, average_length)
    speed = random.uniform(min_speed, max_speed)
    print(speed)
    generator = pipeline(
        text, voice='af_heart', # <= change voice here  af_heart af_jessica
        speed=speed, split_pattern=r'\n+'
    )
    for i, (gs, ps, audio) in enumerate(generator):
        # Assuming 'audio' is your tensor
        audio_tensor = audio.cpu().detach().numpy()
        audio_array = np.squeeze(audio_tensor)

    return audio_array,24000

Helper function to convert the numpy output from the models into audio

In [291]:
import numpy as np
from pydub import AudioSegment
import ast

def numpy_to_audio_segment(audio_arr, sampling_rate):
    """Convert NumPy array to AudioSegment."""
    # Ensure the NumPy array is in the correct format
    if audio_arr.dtype != np.int16:
        # Scale and convert the float array to int16
        audio_arr = (audio_arr * 32767).astype(np.int16)
    
    # Create an AudioSegment instance from the raw data
    audio_segment = AudioSegment(
        audio_arr.tobytes(), 
        frame_rate=sampling_rate,
        sample_width=audio_arr.dtype.itemsize, 
        channels=1
    )
    
    return audio_segment


In [294]:
import re

# Assuming response['message']['content'] contains your string
content_string = PODCAST_TEXT

# Define a regular expression pattern to match the tuples
pattern = r'\("([^"]+)", "([^"]+)"\)'

# Find all matches in the string
matches = re.findall(pattern, content_string)

# Convert matches to a list of tuples
podcast_text = [(speaker, text) for speaker, text in matches]

# Now you can iterate over the list and access each tuple
for speaker, text in podcast_text:
    print(f"{speaker}: {text}")

Speaker 1: Welcome back to another episode where we delve into the captivating realm of artificial intelligence — Knowledge Distillation. Imagine teaching your brainy buddy everything they know, but this time, it's all about bots and coding instead of conversations and hugs! I'm thrilled to be joined by a fellow AI enthusiast who's eager to learn more about this thrilling topic.
Speaker 2: [Excited] Hi there! So, what exactly is Knowledge Distillation? Sounds like some sort of robot secret society!
Speaker 1: Welcome aboard, buddy! It's actually the process of taking advanced capabilities from premium Large Language Models (LLMs) — think top-of-the-line AI brains that cost an arm and a leg — and condensing their knowledge into more accessible open-source models.
Speaker 2: Oh wow, so these LLMs are like genius AI beings? But they're pricey and exclusive?
Speaker 1: Precisely! Models like GPT-4 or Gemini are loaded with incredible problem-solving skills, but they come at a high cost. Th

Most of the times we argue in life that Data Structures isn't very useful. However, this time the knowledge comes in handy. 

We will take the string from the pickle file and load it in as a Tuple with the help of `ast.literal_eval()`

In [295]:
from collections import defaultdict


# Initialize a dictionary to store lengths of text segments per speaker
speaker_lengths = defaultdict(list)

# Populate the dictionary with lengths of each text segment
for speaker, text in podcast_text:
    speaker_lengths[speaker].append(len(text))

# Calculate and display the length of each text segment and the average length per speaker
for speaker, lengths in speaker_lengths.items():
    print(f"\n{speaker}:")
    for i, length in enumerate(lengths, 1):
        print(f"  Text segment {i} length: {length} characters")
    average_length = sum(lengths) / len(lengths) if lengths else 0
    print(f"  Average length of text segments: {average_length:.2f} characters")



Speaker 1:
  Text segment 1 length: 370 characters
  Text segment 2 length: 257 characters
  Text segment 3 length: 265 characters
  Text segment 4 length: 162 characters
  Text segment 5 length: 189 characters
  Text segment 6 length: 312 characters
  Text segment 7 length: 229 characters
  Text segment 8 length: 216 characters
  Text segment 9 length: 223 characters
  Text segment 10 length: 146 characters
  Text segment 11 length: 201 characters
  Text segment 12 length: 207 characters
  Average length of text segments: 231.42 characters

Speaker 2:
  Text segment 1 length: 110 characters
  Text segment 2 length: 82 characters
  Text segment 3 length: 99 characters
  Text segment 4 length: 72 characters
  Text segment 5 length: 31 characters
  Text segment 6 length: 41 characters
  Text segment 7 length: 32 characters
  Text segment 8 length: 101 characters
  Text segment 9 length: 103 characters
  Text segment 10 length: 62 characters
  Text segment 11 length: 70 characters
  Aver

#### Generating the Final Podcast

Finally, we can loop over the Tuple and use our helper functions to generate the audio

In [297]:
final_audio = None

# Calculate the length of each text segment
lengths = [len(text) for _, text in podcast_text]

# Compute the average length
average_length = sum(lengths) / len(lengths) if lengths else 0


for speaker, text in tqdm(podcast_text, desc="Generating podcast segments", unit="segment"):
    if speaker == "Speaker 1":
        audio_arr, rate = generate_speaker1_audio(text,average_length)
    else:  # Speaker 2
        audio_arr, rate = generate_speaker2_audio(text,average_length)
    
    # Convert to AudioSegment (pydub will handle sample rate conversion automatically)
    audio_segment = numpy_to_audio_segment(audio_arr, rate)
    
    # Add to final audio
    if final_audio is None:
        final_audio = audio_segment
    else:
        final_audio += audio_segment

Generating podcast segments:   0%|                  | 0/23 [00:00<?, ?segment/s]

1.025945669726715


Generating podcast segments:   4%|▍         | 1/23 [00:04<01:29,  4.07s/segment]

0.8792193289189597


Generating podcast segments:   9%|▊         | 2/23 [00:05<00:54,  2.62s/segment]

...

Generating podcast segments:  91%|████████▏| 21/23 [00:36<00:03,  1.58s/segment]

0.9213531640794541


Generating podcast segments:  96%|████████▌| 22/23 [00:37<00:01,  1.37s/segment]

1.1042010471522914


Generating podcast segments: 100%|█████████| 23/23 [00:39<00:00,  1.71s/segment]


### Output the Podcast

We can now save this as a mp3 file

In [298]:
final_audio.export("./resources2/_podcast2.mp3", 
                  format="mp3", 
                  bitrate="192k",
                  parameters=["-q:a", "0"])

<_io.BufferedRandom name='./resources2/_podcast2.mp3'>

### Suggested Next Steps:

- Experiment with the prompts: Please feel free to experiment with the SYSTEM_PROMPT in the notebooks
- Extend workflow beyond two speakers
- Test other TTS Models
- Experiment with Speech Enhancer models as a step 5.

In [174]:
#fin